# Doc2Vec

### 위키+뉴스+30벡터

In [ ]:
import gensim

model = gensim.models.Doc2Vec.load("wiki_news_30/all_terms_30vectors")

In [ ]:
from pprint import pprint
pprint(model.most_similar(u'', topn=20))

In [ ]:
pprint(model.most_similar(positive=['여자', '왕'], negative=['남자']))

### 위키+뉴스+100벡터

In [ ]:
import gensim

model = gensim.models.Doc2Vec.load("wiki_news_100/all_terms_50vectors")

In [ ]:
pprint(model.most_similar(positive=['여자', '왕'], negative=['남자']))

# 이게 제일 학습 잘 된듯

### 위키+1000벡터

In [ ]:
import gensim

model = gensim.models.KeyedVectors.load_word2vec_format("wiki_1000/wiki_dmpv_1000_no_taginfo_word2vec_format.bin", binary = True)

In [ ]:
pprint(model.most_similar(positive=['여자', '왕'], negative=['남자']))  # 여왕
pprint(model.most_similar(positive=['한국', '도쿄'], negative=['서울'])) # 일본
pprint(model.most_similar(positive=['한국', '대마도'], negative=['제주도'])) # 일본
# pprint(model.most_similar(positive=['김정은', '한국'], negative=['북한'])) # 근데 인식 잘 안됨

# Online Learning시킬 뉴스데이터 전처리

In [ ]:
from eunjeon import Mecab
from konlpy.tag import Twitter
twitter = Twitter()
mecab = Mecab()

def read_data(filename):
    with open(filename, 'r', encoding = 'utf-8') as f:
        data =[line.split('\t') for line in f.read().splitlines()]
    from random import randint
    random_data = [data[randint(1, len(data))] for no in range(int(len(data)/10))]
    return random_data

def tokenize(doc):
    return ['/'.join(t) for t in mecab.pos(doc)]

In [ ]:
maptrain_data[1]

In [ ]:
from collections import namedtuple
train_data = read_data('news_all.txt')

In [ ]:
for row in train_data[1:5]:
    print(row)

### map (함수: 반복 가능한 자료형)
- https://wayhome25.github.io/cs/2017/04/03/cs-03/
- 리스트를 다룰 땐, 원소 갯수를 고려해 indexing을 해야한다.! 남의 코드 생각없이 넣으면 큰일.

In [ ]:
# for문 사용
# train_docs = [(tokenize(row[0])) for row in train_data[1:5]]

# 람다와 맵 사용
train_docs = list(map(lambda x: tokenize(x[0]), train_data))

### mecab 형태소 태그 설명
- http://openuiz.blogspot.com/2016/07/mecab-ko-dic.html

In [ ]:
train_docs[-1:]

In [ ]:
len(train_docs)

In [ ]:
a = '그래 니 똥 굵다.'
print(twitter.pos(a))
print(mecab.pos(a))

# Online Learning 진행
- 은 online training 기능 지원 안함~ 수고~
- https://stackoverflow.com/questions/53616003/doc2vec-online-training

In [ ]:
%%time
from gensim.models import doc2vec
doc_vectorizer = doc2vec.Doc2Vec(vector_size = 300, alpha = 0.025, min_alpha = 0.025, seed = 1234)
doc_vectorizer.build_vacab(tagged_train_docs)

for epoch in range(10):
    doc_vectorizer.train(train_docs,
                        total_examples = doc_vectorizer.corpus_count,
                        epochs = doc_vectorizer.epochs)
    doc_vectorizer.alpha -= 0.002
    doc_vectorizer.min_alpha = doc_vectorizer.alpha
    
# 학습이 완료된 모델 데이터 저장
doc_vectorizer.save('doc2vec_sample.model')

# wiki 원본데이터 임포트
- pos tagging형태가 다를 수 있음... 후...
- 일단은 txt만 임포트해서 전처리 후 합치기 (1번+3번)

In [ ]:
with open("1.wiki_data_mine.txt" , "r", encoding = 'utf-8') as bigfile:
    reader_original = bigfile.read()

In [ ]:
reader_original[:10]

In [ ]:
import codecs

tagger = Twitter()
corpus = codecs.open('corpus.txt', 'w', encoding='utf-8')

def flat(content):
    return ["{}/{}".format(word, tag) for word, tag in tagger.pos(content)]

corpus.write(' '.join(flat("메이드복 입은 제이미 귀엽다")) + '\n')
# 메이드/Noun 복/Noun 입은/Verb 제이/Noun 미/Adjective 귀엽/Adjective 다/Eomi

In [ ]:
class SentenceReader:

    def __init__(self, filepath):
        self.filepath = filepath

    def __iter__(self):
        for line in codecs.open(self.filepath, encoding='utf-8'):
            yield line.split(' ')

In [ ]:
sentences_vocab = SentenceReader('2.wiki_data_pos_tokenizer_with_tag.txt')
sentences_train = SentenceReader('corpus.txt')

model = gensim.models.Word2Vec()
model.build_vocab(sentences_vocab)
model.train(sentences_train)

In [ ]:
# 겐심 튜토리얼

In [ ]:
with open("1.wiki_data_mine.txt" , "r", encoding = 'utf-8') as bigfile:
    test1 = bigfile.read()

In [3]:
test1 = ''

In [ ]:
with open("1.wiki_data_mine.txt" , "r", encoding = 'utf-8') as bigfile:
    for line in bigfile:
        test1 += line

In [1]:
with open("3.news_all.txt" , "r", encoding = 'utf-8') as bigfile:
    test2 = bigfile.read()

In [ ]:
import pandas as pd